# JWT スコープを使用した AgentCore Gateway インターセプターによるきめ細かいアクセス制御

## 概要

このノートブックでは、**Gateway インターセプター**と **JWT スコープ**を使用して **AgentCore Gateway** で**きめ細かいアクセス制御（FGAC）**を実施する方法を示します。目標は、MCP ターゲットが提供するツールの数に関係なく、エージェントエンドポイントを保護するための再利用可能なパターンを提供することです。

### なぜこれが重要か

エージェントが拡大するにつれて、以下のことが必要になる場合があります：

- 特定のユーザーが呼び出せる**ツール**を制限する
- **機密性の高いアクション**（cancelOrder、updateOrder、deleteOrder など）へのアクセスを制御する
- ユーザー権限に基づいて**セマンティック検索結果**をフィルタリングまたは編集する
- ユーザーに**許可されたツールのみを表示する**
- JWT トークンが提供するものを超えるカスタム認可ロジックを実施する
- 個々のツールやランタイムを変更せずに**一元化されたガバナンス**を適用する

Gateway インターセプターは、**エージェント、ランタイム、または MCP サーバーを変更せずに**これらの制御を実装するためのスケーラブルでプラグアンドプレイの方法を提供します。
すべてのリクエストが自然に流れる **Gateway レベル**でポリシーを実施します。

---

## このチュートリアルでカバーする内容

3 つの主要な Gateway 操作にわたって FGAC を実装します：

1. 🛠️ **FGAC によるツール呼び出し（リクエスト Gateway インターセプター）**
   Gateway インターセプターを使用して `tools/call` でツールごとの権限を実施します。
   ![invoke tool](images/invoke-tool.png)

2. 🔍 **FGAC によるセマンティック検索（レスポンス Gateway インターセプター）**
   Cognito スコープに基づいて検索結果をフィルタリングし、ユーザーが許可されたデータのみを表示できるようにします。
   ![search tool](images/search-tool.png)

3. 📋 **FGAC によるツール一覧表示（レスポンス Gateway インターセプター）**
   ユーザーが認可されたツールのみを表示するように、ツールカタログを動的にフィルタリングします。
   ![list tool](images/list-tool.png)

---

## なぜ Gateway インターセプターを使用するのか？

Gateway インターセプターにより、以下のことが可能になります：

- **きめ細かいアクセス制御の実装**
  ユーザーごと、ツールごと、アクションごとの認可ルールを実施します。

- **カスタム認可ロジックの注入**
  動的ルールや外部ポリシーで静的な JWT 検証を超えます。

- **監査とガバナンス**
  ツール使用の試行をログに記録し、コンプライアンスの可視性を提供します。

- **リクエスト/レスポンス変換**
  ユーザーに表示される前にデータを編集したり、リクエストを変更したり、レスポンスをフィルタリングしたりします。

インターセプターは **Gateway レイヤー**にアタッチされているため、**任意の**基盤となる MCP サーバーまたはランタイムに対して中央ポリシーを実施します。

---

## チュートリアルの詳細

| 情報                     | 詳細                                                                                            |
|--------------------------|-------------------------------------------------------------------------------------------------|
| **チュートリアルタイプ** | インタラクティブ                                                                                |
| **AgentCore コンポーネント** | AgentCore Gateway、Gateway インターセプター                                                  |
| **Gateway ターゲットタイプ** | MCP サーバー（AgentCore Runtime 上で実行される FastMCP）                                     |
| **インターセプタータイプ** | AWS Lambda（リクエスト + レスポンス）                                                         |
| **インバウンド認証 IdP** | Amazon Cognito（CUSTOM\_JWT オーソライザー）                                                   |
| **アクセス制御**         | Cognito スコープ + Gateway インターセプターを使用した FGAC                                     |
| **チュートリアルコンポーネント** | Gateway、Runtime MCP サーバー、Cognito、Gateway インターセプター、MCP ツール              |
| **チュートリアル分野**   | クロスバーティカル                                                                              |
| **難易度**               | 簡単～中級                                                                                      |
| **使用 SDK**             | boto3                                                                                           |

---

## 前提条件

このチュートリアルを実行するには、以下が必要です：

- Jupyter notebook（Python カーネル）
- 以下の権限を持つ AWS 資格情報：
  - Lambda
  - IAM
  - Cognito
  - DynamoDB（ここでは使用しませんが、実際の FGAC では役立つことが多い）
  - AgentCore サービス（コントロールプレーン + ランタイム）
- Python 3.13 以上
- AWS Lambda、IAM ロール、Cognito、AgentCore Gateway の基本的な理解

> ⚠️ **注意：** 最後のクリーンアップセクションは、このチュートリアルで作成された AWS リソース（Gateway、Lambda、IAM ロールなど）を削除します。すべてを削除する準備ができたときにのみ実行してください。

In [ ]:
# 現在のディレクトリの requirements ファイルからインストール
!pip install -r requirements.txt --quiet

In [ ]:
# 必要なライブラリをインポートし、リソース命名用の一意のタイムスタンプを生成

import boto3
import json
import time
import zipfile
import subprocess
import tempfile
import io
import os
import sys
import requests
import uuid
from datetime import datetime

from botocore.exceptions import ClientError

print("✓ ライブラリをインポートしました")

# 一意の命名用タイムスタンプを生成
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
print(f"タイムスタンプ: {timestamp}")

# MCP ターゲット名を定義
gateway_target_name = f"mcp-target-{timestamp}"

# AWS 認証情報とリージョンを設定
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')
REGION = os.environ['AWS_DEFAULT_REGION']

print(f"リージョン: {REGION}")

In [ ]:
# ユーティリティをインポートしてロギングを設定

# 現在のスクリプトのディレクトリを取得
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック（例：Jupyter）

# utils.py を含むディレクトリに移動（1つ上の階層）
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# sys.path に追加
sys.path.insert(0, utils_dir)

# utils をインポート
import utils

# ロギングをセットアップ
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

logging.getLogger("strands").setLevel(logging.INFO)

print("✓ ロギングを設定し、utils をインポートしました")

## パート 1: インバウンド認証 – Gateway 用の Cognito ユーザープールとスコープ

まず、FGAC に使用されるスコープを定義する **Cognito ユーザープール**と**リソースサーバー**を作成します：

- MCP ターゲットへの**フルアクセス**用のスコープ
- **個々のツール**用のスコープ：`getOrder`、`updateOrder`、`cancelOrder`、`deleteOrder`

これらのスコープはアクセストークンに埋め込まれ、以下の両方で使用されます：

- **リクエスト Gateway インターセプター**（ツール呼び出し用）
- **レスポンス Gateway インターセプター**（セマンティック検索と tools/list 用）

In [ ]:
# Gateway へのインバウンド認可用の Amazon Cognito プールを作成

# 各実行で新しいプール/リソースサーバー/クライアントを取得するためにタイムスタンプ付きの名前を使用
USER_POOL_NAME = f"gateway-interceptor-pool-{timestamp}"
RESOURCE_SERVER_ID = f"gateway-interceptor-id-{timestamp}"
RESOURCE_SERVER_NAME = "gateway-interceptor-name"
CLIENT_NAME = f"gateway-interceptor-client-{timestamp}"

# スコープは今回の実行の gateway_target_name に基づく
SCOPES = [
    # MCP ターゲットへのフルアクセス
    {
        "ScopeName": gateway_target_name,
        "ScopeDescription": "Full access to all tools in MCP target"
    },

    # 特定のツールへのアクセス
    {
        "ScopeName": f"{gateway_target_name}:getOrder",
        "ScopeDescription": "Access to getOrder tool only"
    },
    {
        "ScopeName": f"{gateway_target_name}:updateOrder",
        "ScopeDescription": "Access to updateOrder tool only"
    },
    {
        "ScopeName": f"{gateway_target_name}:cancelOrder",
        "ScopeDescription": "Access to cancelOrder tool only"
    },
    {
        "ScopeName": f"{gateway_target_name}:deleteOrder",
        "ScopeDescription": "Access to deleteOrder tool only"
    },
]

# Cognito 形式の完全なスコープ文字列: "<resource-server-id>/<scope-name>"
scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
scopeString = " ".join(scope_names)

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognito リソースを作成または取得中...")
gw_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"ユーザープール ID: {gw_user_pool_id}")

utils.get_or_create_resource_server(
    cognito,
    gw_user_pool_id,
    RESOURCE_SERVER_ID,
    RESOURCE_SERVER_NAME,
    SCOPES
)
print("リソースサーバーを確認しました。")

gw_client_id, gw_client_secret = utils.get_or_create_m2m_client(
    cognito,
    gw_user_pool_id,
    CLIENT_NAME,
    RESOURCE_SERVER_ID,
    scope_names
)
print(f"クライアント ID: {gw_client_id}")

# Gateway オーソライザーと utils.get_token で後ほど使用するディスカバリー URL
gw_cognito_discovery_url = (
    f"https://cognito-idp.{REGION}.amazonaws.com/{gw_user_pool_id}/.well-known/openid-configuration"
)
gw_jwks_url = f"https://cognito-idp.{REGION}.amazonaws.com/{gw_user_pool_id}/.well-known/jwks.json"
print(gw_cognito_discovery_url)

## パート 2: Gateway インターセプター用の IAM ロール
両方の Gateway インターセプター（リクエストとレスポンス）が使用する単一の IAM ロールを作成します。

In [ ]:
# Gateway インターセプター用の IAM ロールを作成

iam_client = boto3.client('iam', region_name=REGION)

lambda_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
    
lambda_role_name = f'GatewayInterceptorRole-{timestamp}'

lambda_role_response = iam_client.create_role(
    RoleName=lambda_role_name,
    AssumeRolePolicyDocument=json.dumps(lambda_trust_policy),
    Description='IAM role for Gateway Interceptors'
)

# 基本的な Lambda 実行ポリシーをアタッチ
iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

lambda_role_arn = lambda_role_response['Role']['Arn']
print(f"Lambda IAM ロールを作成しました: {lambda_role_arn}")

# ロールが利用可能になるまで待機
print("Lambda ロールが利用可能になるまで待機中...")
time.sleep(10)
print("✓ Lambda ロール準備完了")

## パート 3: ツール呼び出し用のリクエスト Gateway インターセプター

このインターセプターは：

- Authorization ヘッダーを検証
- JWT をデコードして `scope` クレームを読み取る
- MCP `tools/call` リクエストからツール名を抽出
- トークンのスコープがそのツールの呼び出しを許可しているかチェック
- 以下を返す：
  - **許可されていない**場合は 403 エラー MCP レスポンス
  - **許可されている**場合はパススルーリクエスト

これにより**呼び出し時の FGAC** が提供されます。

In [ ]:
# ツール呼び出し FGAC 用のリクエスト Gateway インターセプター関数を作成

def create_request_gateway_interceptor():
    """リクエスト Gateway インターセプター関数を作成"""
    lambda_client = boto3.client('lambda', region_name=REGION)
   
    lambda_code = '''import json
import time
import urllib.request
from jose import jwk, jwt
from jose.utils import base64url_decode

# Gateway ターゲット名とリソースサーバー ID を定義
gateway_target_name = "TARGET_GATEWAY_NAME_PLACEHOLDER"
resource_server_id = "RESOURCE_SERVER_ID_PLACEHOLDER"
jwks_url = "JWKS_URL_PLACEHOLDER"
client_id = "CLIENT_ID_PLACEHOLDER"

with urllib.request.urlopen(jwks_url) as f:
    response = f.read()
keys = json.loads(response.decode('utf-8'))['keys']

# リクエストインターセプターで FGAC によってブロックすべきでないシステム/内部ツール。
# これらは代わりにレスポンスインターセプターでフィルタリングされます（必要に応じて）。
SYSTEM_TOOLS = {
    "x_amz_bedrock_agentcore_search",  # セマンティック検索ツール
}

def decode_jwt_payload(token):
    """JWT ペイロードを検証してデコード"""
    headers = jwt.get_unverified_headers(token)
    kid = headers['kid']
    
    # 一致するキーを検索
    key_index = -1
    for i, key in enumerate(keys):
        if kid == key['kid']:
            key_index = i
            break
    
    if key_index == -1:
        raise Exception('Public key not found in jwks.json')
    
    # 公開鍵を構築
    public_key = jwk.construct(keys[key_index])
    
    # メッセージと署名を取得
    message, encoded_signature = str(token).rsplit('.', 1)
    decoded_signature = base64url_decode(encoded_signature.encode('utf-8'))
    
    # 署名を検証
    if not public_key.verify(message.encode("utf8"), decoded_signature):
        raise Exception('Signature verification failed')
    
    # クレームを取得
    claims = jwt.get_unverified_claims(token)
    
    # 有効期限をチェック
    if time.time() > claims['exp']:
        raise Exception('Token is expired')
    
    # client_id をチェック
    if claims['client_id'] != client_id:
        raise Exception('Token was not issued for this audience')
    
    # トークン使用をチェック
    if claims.get('token_use') != 'access':
        raise Exception("Invalid token use: must be 'access' token")
    
    return claims

def check_tool_authorization(scopes, tool_name):
    """
    スコープに基づいてユーザーが特定のツールに対する権限を持っているかチェック。

    Cognito はスコープを以下の形式で返します：
      "<resource_server_id>/<actual_scope>"
    例：
      "agentcore-gateway-interceptor-id-20251119-141611/mcp-target-20251119-141611:getOrder"

    スラッシュの後の部分のみを使用：
      "mcp-target-20251119-141611:getOrder"
    """
    if not scopes:
        return False
    
    user_scopes = scopes.split(" ")
    actual_scopes = []
    
    for s in user_scopes:
        # リソースサーバープレフィックスがある場合は削除
        if "/" in s:
            actual_scopes.append(s.split("/", 1)[1])
        else:
            actual_scopes.append(s)

    # この MCP ターゲット内のすべてのツールへのフルアクセス
    if gateway_target_name in actual_scopes:
        return True
    
    # 特定のツール権限: <target>:<toolName>
    required_scope = f"{gateway_target_name}:{tool_name}"
    if required_scope in actual_scopes:
        return True
    
    return False

def extract_tool_name(body):
    """MCP tools/call リクエストボディからツール名を抽出"""
    try:
        if isinstance(body, dict):
            params = body.get("params", {})
            tool_name = params.get("name", "")
            # ツール名は <target>___<toolName> の形式
            if "___" in tool_name:
                return tool_name.split("___")[-1]
            return tool_name
    except Exception:
        pass
    return None

def build_pass_through_response(auth_header, body):
    """リクエストをターゲットに到達させるパススルーレスポンスを返す"""
    return {
        "interceptorOutputVersion": "1.0",
        "mcp": {
            "transformedGatewayRequest": {
                "headers": {
                    "Authorization": auth_header,
                    "Content-Type": "application/json"
                },
                "body": body
            }
        }
    }

def build_error_response(message, body, status_code=403):
    """MCP 形式のエラーレスポンスを返す"""
    return {
        "interceptorOutputVersion": "1.0",
        "mcp": {
            "transformedGatewayResponse": {
                "statusCode": status_code,
                "body": {
                    "jsonrpc": "2.0",
                    "id": body.get("id", "unknown") if isinstance(body, dict) else "unknown",
                    "error": {
                        "code": -32600,
                        "message": message
                    }
                }
            }
        }
    }

def lambda_handler(event, context):
    print(f"Received event: {json.dumps(event)}")
    
    # 正しい構造から Gateway リクエストを抽出
    mcp_data = event.get("mcp", {})
    gateway_request = mcp_data.get("gatewayRequest", {})
    headers = gateway_request.get("headers", {})
    body = gateway_request.get("body", {})
    
    # Authorization ヘッダーを抽出
    auth_header = headers.get("Authorization", "")
    
    # すべてのリクエストに Bearer トークンの存在を強制
    if not auth_header.startswith("Bearer "):
        response = build_error_response("No authorization token provided", body)
        print(f"Returning error response (no token): {json.dumps(response)}")
        return response
    
    # トークンとスコープをデコード
    try:
        token = auth_header.replace("Bearer ", "")
        decoded_token = decode_jwt_payload(token)
        scopes = decoded_token.get("scope", "")
        
        method = body.get("method", "")
        tool_name = extract_tool_name(body)
        
        print(f"Decoded scopes (raw): {scopes}")
        print(f"MCP method: {method}")
        print(f"Requested tool: {tool_name}")
        
        # 1) tools/list を許可: 表示されるツールの FGAC は
        #    レスポンス Gateway インターセプターで実施（返されるリストをフィルタリング）
        if method == "tools/list":
            print("tools/list request detected - skipping tool-level FGAC in request gateway interceptor")
            response = build_pass_through_response(auth_header, body)
            print(f"Returning pass-through response for tools/list: {json.dumps(response)}")
            return response
        
        # 2) システムツール（例：x_amz_bedrock_agentcore_search）はパススルーを許可。
        #    レスポンス Gateway インターセプターが後でツールリスト/検索結果をフィルタリング。
        if tool_name in SYSTEM_TOOLS:
            print(f"System tool '{tool_name}' detected - skipping tool-level FGAC in request gateway interceptor")
            response = build_pass_through_response(auth_header, body)
            print(f"Returning pass-through response for system tool: {json.dumps(response)}")
            return response
        
        # 3) 他のすべてのツール（getOrder、updateOrder などのビジネスツール）には
        #    スコープベースの FGAC を適用。
        if not tool_name:
            response = build_error_response("No tool name provided in request", body)
            print(f"Returning error response (no tool name): {json.dumps(response)}")
            return response
        
        if not check_tool_authorization(scopes, tool_name):
            response = build_error_response(
                f"Insufficient permission for tool: {tool_name}",
                body
            )
            print(f"Returning error response (FGAC deny): {json.dumps(response)}")
            return response
    
    except Exception as e:
        print(f"Error while validating token/scopes: {e}")
        response = build_error_response(f"Invalid token - {e}", body)
        print(f"Returning error response (exception): {json.dumps(response)}")
        return response
    
    # 認可済み → パススルー
    response = build_pass_through_response(auth_header, body)
    print(f"Returning pass-through response (authorized): {json.dumps(response)}")
    return response
'''
    # プレースホルダーを実際の値に置換
    lambda_code = lambda_code.replace("TARGET_GATEWAY_NAME_PLACEHOLDER", gateway_target_name)
    lambda_code = lambda_code.replace("RESOURCE_SERVER_ID_PLACEHOLDER", RESOURCE_SERVER_ID)
    lambda_code = lambda_code.replace("JWKS_URL_PLACEHOLDER", gw_jwks_url)
    lambda_code = lambda_code.replace("CLIENT_ID_PLACEHOLDER", gw_client_id)
    
    # 依存関係を含む Lambda 用の ZIP ファイルを作成
    with tempfile.TemporaryDirectory() as temp_dir:
        # 依存関係をインストール
        subprocess.run(['pip', 'install', '-r', 'requirements_lambda.txt', '-t', temp_dir], check=True)
        
        # ZIP バッファを作成
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            # Lambda 関数コードを追加
            zip_file.writestr('lambda_function.py', lambda_code)
            
            # すべての依存関係ファイルを追加
            for root, dirs, files in os.walk(temp_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arc_name = os.path.relpath(file_path, temp_dir)
                    zip_file.write(file_path, arc_name)
    
    zip_buffer.seek(0)
    
    # Lambda 関数を作成
    lambda_function_name = f'request-gateway-interceptor-{timestamp}'
    
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_function_name,
        Runtime='python3.13',
        Role=lambda_role_arn,
        Handler='lambda_function.lambda_handler',
        Code={'ZipFile': zip_buffer.read()},
        Description='Request Gateway Interceptor for AgentCore Gateway - Tool Invocation FGAC'
    )
    
    request_lambda_arn = lambda_response['FunctionArn']
    print(f"リクエスト Gateway インターセプター関数を作成しました: {request_lambda_arn}")
    
    return request_lambda_arn

request_lambda_arn = create_request_gateway_interceptor()
print(f"\n✅ リクエスト Gateway インターセプターの作成が完了しました: {request_lambda_arn}")

## パート 4: セマンティック検索と Tools/List 用のレスポンス Gateway インターセプター

このインターセプターは：

- レスポンスの Authorization ヘッダーから JWT スコープをデコード
- 許可されたスコープに基づいてツールリストをフィルタリング
- 以下に適用：
  - セマンティック検索結果（`x_amz_bedrock_agentcore_search`）
  - MCP `tools/list` レスポンス（純粋なツール列挙）

ロジック：

- スコープが MCP ターゲット名と等しい場合 → そのターゲットからのすべてのツールへのフルアクセス
- スコープが `mcpTarget:toolName` と等しい場合 → その特定のツールへのアクセスのみ
- `target___toolName` の命名規則に従わない内部/システムツールをスキップ

In [ ]:
def create_response_gateway_interceptor():
    """レスポンス Gateway インターセプター関数を作成"""
    lambda_client = boto3.client('lambda', region_name=REGION)
   
    lambda_code = '''import json
import time
import urllib.request
from jose import jwk, jwt
from jose.utils import base64url_decode

# Gateway ターゲット名とリソースサーバー ID を定義
jwks_url = "JWKS_URL_PLACEHOLDER"
client_id = "CLIENT_ID_PLACEHOLDER"

with urllib.request.urlopen(jwks_url) as f:
    response = f.read()
keys = json.loads(response.decode('utf-8'))['keys']

def decode_jwt_payload(token):
    """JWT ペイロードを検証してデコード"""
    headers = jwt.get_unverified_headers(token)
    kid = headers['kid']
    
    # 一致するキーを検索
    key_index = -1
    for i, key in enumerate(keys):
        if kid == key['kid']:
            key_index = i
            break
    
    if key_index == -1:
        raise Exception('Public key not found in jwks.json')
    
    # 公開鍵を構築
    public_key = jwk.construct(keys[key_index])
    
    # メッセージと署名を取得
    message, encoded_signature = str(token).rsplit('.', 1)
    decoded_signature = base64url_decode(encoded_signature.encode('utf-8'))
    
    # 署名を検証
    if not public_key.verify(message.encode("utf8"), decoded_signature):
        raise Exception('Signature verification failed')
    
    # クレームを取得
    claims = jwt.get_unverified_claims(token)
    
    # 有効期限をチェック
    if time.time() > claims['exp']:
        raise Exception('Token is expired')
    
    # client_id をチェック
    if claims['client_id'] != client_id:
        raise Exception('Token was not issued for this audience')
    
    # トークン使用をチェック
    if claims.get('token_use') != 'access':
        raise Exception("Invalid token use: must be 'access' token")
    
    return claims

def filter_tools_by_scope(tools, allowed_scopes):
    """カスタムスコープに基づいてツールをフィルタリング"""
    if not allowed_scopes:
        return []
    
    filtered_tools = []
    for tool in tools:
        tool_name = tool.get("name", "")
        
        # ターゲットセパレーターのないシステム生成 MCP ツールをスキップ
        if "___" not in tool_name:
            continue
            
        mcp_target = tool_name.split("___")[0]
        tool_action = tool_name.split("___")[1]
        
        for scope in allowed_scopes:
            # リソースサーバープレフィックスを削除して実際のスコープを取得
            actual_scope = scope.split("/")[-1] if "/" in scope else scope
            
            # MCP ターゲットへのフルアクセス
            if actual_scope == mcp_target:
                filtered_tools.append(tool)
                break
            # 特定のツールへのアクセス
            elif actual_scope == f"{mcp_target}:{tool_action}":
                filtered_tools.append(tool)
                break
    
    return filtered_tools

def lambda_handler(event, context):
    print(f"Received event: {json.dumps(event)}")
    
    # Gateway レスポンスを抽出
    mcp_data = event.get("mcp", {})
    gateway_response = mcp_data.get("gatewayResponse", {})
    headers = gateway_response.get("headers", {})
    body = gateway_response.get("body", {})

    # Authorization ヘッダーをパススルー
    auth_header = headers.get("Authorization", "")
    token = auth_header.replace("Bearer ", "") if auth_header.startswith("Bearer ") else ""

    try:
        # 検証なしで JWT ペイロードをパース
        claims = decode_jwt_payload(token)
        
        # クレームからスコープを抽出
        scope_string = claims.get("scope", "")
        scopes = scope_string.split() if scope_string else []
        print(f"Extracted scopes: {scopes}")
        
        # Gateway レスポンスからツールを取得（セマンティック検索と tools/list をサポート）
        result = body.get("result", {})
        tools = result.get("tools", [])
        if not tools:
            structured_content = result.get("structuredContent", {})
            tools = structured_content.get("tools", [])
        print(f"Available tools: {[tool.get('name') for tool in tools]}")
        
        # スコープに基づいてツールをフィルタリング
        filtered_tools = filter_tools_by_scope(tools, scopes)
        print(f"Filtered tools: {[tool.get('name') for tool in filtered_tools]}")
        
        # フィルタリングされたツールでボディを更新
        filtered_body = body.copy()
        if "result" in filtered_body:
            if "structuredContent" in filtered_body["result"]:
                filtered_body["result"]["structuredContent"]["tools"] = filtered_tools
            else:
                filtered_body["result"]["tools"] = filtered_tools
            
            # content 配列にテキスト内に埋め込まれたツール JSON が含まれている場合はフィルタリング
            if "content" in filtered_body["result"]:
                for content_item in filtered_body["result"]["content"]:
                    if content_item.get("type") == "text" and "text" in content_item:
                        try:
                            content_data = json.loads(content_item["text"])
                            if "tools" in content_data:
                                content_filtered_tools = filter_tools_by_scope(content_data["tools"], scopes)
                                content_data["tools"] = content_filtered_tools
                                content_item["text"] = json.dumps(content_data)
                        except (json.JSONDecodeError, KeyError):
                            pass
    except Exception as e:
        print(f"Error processing JWT or filtering tools: {e}")
        filtered_body = body
    else:
        # except に入らなかった場合のみ filtered_body を設定
        pass

    # filtered_body が try/except で設定されなかった場合は元の body をデフォルトに
    if "filtered_body" not in locals():
        filtered_body = body

    # 変換されたレスポンスを返す
    response = {
        "interceptorOutputVersion": "1.0",
        "mcp": {
            "transformedGatewayResponse" : {
                "statusCode": 200,
                "headers": {
                    "Accept": "application/json",
                    "Authorization": auth_header
                },
                "body": filtered_body
            }
        }   
    }
    
    print(f"Returning response: {json.dumps(response)}")
    return response
'''
    
    # プレースホルダーを実際の値に置換
    lambda_code = lambda_code.replace("JWKS_URL_PLACEHOLDER", gw_jwks_url)
    lambda_code = lambda_code.replace("CLIENT_ID_PLACEHOLDER", gw_client_id)
    
    # 依存関係を含む Lambda 用の ZIP ファイルを作成
    with tempfile.TemporaryDirectory() as temp_dir:
        # 依存関係をインストール
        subprocess.run(['pip', 'install', '-r', 'requirements_lambda.txt', '-t', temp_dir], check=True)
        
        # ZIP バッファを作成
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            # Lambda 関数コードを追加
            zip_file.writestr('lambda_function.py', lambda_code)
            
            # すべての依存関係ファイルを追加
            for root, dirs, files in os.walk(temp_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arc_name = os.path.relpath(file_path, temp_dir)
                    zip_file.write(file_path, arc_name)
    
    zip_buffer.seek(0)
    
    # Lambda 関数を作成
    lambda_function_name = f'response-gateway-interceptor-{timestamp}'
    
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_function_name,
        Runtime='python3.13',
        Role=lambda_role_arn,
        Handler='lambda_function.lambda_handler',
        Code={'ZipFile': zip_buffer.read()},
        Description='Response Gateway Interceptor for AgentCore Gateway - Search & List FGAC'
    )
    
    response_lambda_arn = lambda_response['FunctionArn']
    print(f"レスポンス Gateway インターセプター関数を作成しました: {response_lambda_arn}")
    
    return response_lambda_arn

response_lambda_arn = create_response_gateway_interceptor()
print(f"\n✅ レスポンス Gateway インターセプターの作成が完了しました: {response_lambda_arn}")

## パート 5: 両方のインターセプターを持つ AgentCore Gateway の作成

以下の構成で AgentCore Gateway を作成します：

- **プロトコル:** MCP
- **検索タイプ:** SEMANTIC
- **リクエスト Gateway インターセプター:** ツール呼び出し FGAC
- **レスポンス Gateway インターセプター:** セマンティック検索 & tools/list FGAC
- **オーソライザー:** 上記の Cognito ユーザープールとクライアントを使用した CUSTOM\_JWT

In [ ]:
gateway_role_name = f'BedrockAgentCoreGatewayRole-{timestamp}'
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role(gateway_role_name)
role_arn=agentcore_gateway_iam_role['Role']['Arn']
print("Agentcore gateway role ARN: ", role_arn)

In [ ]:
# boto3 クライアントを使用してインターセプター付きの Gateway を作成
def create_gateway_with_interceptors():
    gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
    
    print("インターセプター付きの Gateway を作成中...")
    gateway_response = gateway_client.create_gateway(
        name=f"gateway-interceptor-{timestamp}",
        roleArn=role_arn,
        protocolType='MCP',
        protocolConfiguration={
            'mcp': {
                'supportedVersions': ['2025-03-26'],
                'searchType': 'SEMANTIC'
            }
        },
        interceptorConfigurations=[
            {
                'interceptor': {
                    'lambda': {
                        'arn': request_lambda_arn
                    }
                },
                'interceptionPoints': ['REQUEST'],
                'inputConfiguration': {
                    'passRequestHeaders': True
                }
            },
            {
                'interceptor': {
                    'lambda': {
                        'arn': response_lambda_arn
                    }
                },
                'interceptionPoints': ['RESPONSE'],
                'inputConfiguration': {
                    'passRequestHeaders': False
                }
            }
        ],
        authorizerType='CUSTOM_JWT',
        authorizerConfiguration={
            'customJWTAuthorizer': {
                'discoveryUrl': gw_cognito_discovery_url,
                'allowedClients': [gw_client_id]
            }
        }
    )
    
    print("Gateway 作成レスポンス:", gateway_response)
    
    gateway_id = gateway_response['gatewayId']
    gateway_url = gateway_response['gatewayUrl']
    print(f"インターセプター付きの Gateway を作成しました: {gateway_id}")
    
    # Gateway が準備完了になるまで待機
    print("Gateway が準備完了になるまで待機中...")
    while True:
        status_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
        current_status = status_response.get('status', 'UNKNOWN')
        print(f"Gateway ステータス: {current_status}")
        if current_status == 'READY':
            print(f"最終 Gateway 詳細: {json.dumps(status_response, indent=2, default=str)}")
            break
        time.sleep(10)
    
    print("Gateway の準備が完了しました")
    return gateway_id, gateway_url

gateway_id, gateway_url = create_gateway_with_interceptors()
print(f"\n✅ Gateway の作成が完了しました: Gateway ID {gateway_id}")
print(f"Gateway URL: {gateway_url}")

## パート 6: サンプル MCP サーバーの作成と AgentCore Runtime でのホスティング

以下を行います：

1. **Runtime 用の Cognito ユーザープール**を作成（Gateway から Runtime へのアウトバウンド認証用）
2. 4 つのツールを持つシンプルな **FastMCP サーバー**を AgentCore Runtime にホスト：
   - `getOrder`
   - `updateOrder`
   - `cancelOrder`
   - `deleteOrder`
3. AgentCore Runtime でホスト

In [ ]:
# Runtime 用の Cognito ユーザープールを作成（Gateway から Runtime へのアウトバウンド認証）

RUNTIME_USER_POOL_NAME = f"gateway-interceptor-rt-pool-{timestamp}"
RUNTIME_RESOURCE_SERVER_ID = f"gateway-interceptor-rt-id-{timestamp}"
RUNTIME_RESOURCE_SERVER_NAME = "gateway-interceptor-rt-name"
RUNTIME_CLIENT_NAME = f"gateway-interceptor-runtime-rt-{timestamp}"

RUNTIME_SCOPES = [
    {
        "ScopeName": "tools",
        "ScopeDescription": "Scope for search,list and invoke the agentcore gateway"
    },
]

runtime_scope_names = [f"{RUNTIME_RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in RUNTIME_SCOPES]
runtimeScopeString = " ".join(runtime_scope_names)

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Runtime 用の Cognito リソースを作成または取得中...")
runtime_user_pool_id = utils.get_or_create_user_pool(cognito, RUNTIME_USER_POOL_NAME)
print(f"Runtime ユーザープール ID: {runtime_user_pool_id}")

utils.get_or_create_resource_server(
    cognito,
    runtime_user_pool_id,
    RUNTIME_RESOURCE_SERVER_ID,
    RUNTIME_RESOURCE_SERVER_NAME,
    RUNTIME_SCOPES
)
print("Runtime リソースサーバーを確認しました。")

runtime_client_id, runtime_client_secret = utils.get_or_create_m2m_client(
    cognito,
    runtime_user_pool_id,
    RUNTIME_CLIENT_NAME,
    RUNTIME_RESOURCE_SERVER_ID,
    runtime_scope_names
)

print(f"Runtime クライアント ID: {runtime_client_id}")

runtime_cognito_discovery_url = (
    f'https://cognito-idp.{REGION}.amazonaws.com/{runtime_user_pool_id}/.well-known/openid-configuration'
)
print("Runtime Cognito ディスカバリー URL:", runtime_cognito_discovery_url)

In [ ]:
# サンプル MCP サーバーファイル（FastMCP）を作成し、AgentCore Runtime でホスト

content = """
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def getOrder() -> int:
    '''注文を取得'''
    return 123

@mcp.tool()
def updateOrder(orderId: int) -> int:
    '''既存の注文を更新'''
    return 456

@mcp.tool()
def cancelOrder(orderId: int) -> int:
    '''既存の注文をキャンセル'''
    return 789

@mcp.tool()
def deleteOrder(orderId: int) -> int:
    '''既存の注文を削除'''
    return 101

if __name__ == "__main__":
    mcp.run(transport="streamable-http")
"""

with open("mcp_server.py", "w") as f:
    f.write(content)

In [ ]:
# MCP サーバーを設定して AgentCore Runtime にデプロイ

from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
print(f"AWS リージョン: {REGION}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"必要なファイル {file} が見つかりません")
print("すべての必要なファイルが見つかりました ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [runtime_client_id],
        "discoveryUrl": runtime_cognito_discovery_url
    }
}

print("AgentCore Runtime を設定中...")
runtime_response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=REGION,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=f"ac_gateway_mcp_server"
)
print("設定が完了しました ✓")

In [ ]:
# MCP サーバーを AgentCore Runtime に起動

print("MCP サーバーを AgentCore Runtime に起動中...")
launch_result = agentcore_runtime.launch()

runtime_agent_arn = launch_result.agent_arn
runtime_agent_id = launch_result.agent_id

encoded_arn = runtime_agent_arn.replace(":", "%3A").replace("/", "%2F")

agent_url = (
    f'https://bedrock-agentcore.{REGION}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT'
)
print("起動が完了しました ✓")
print(f"エージェント ARN: {runtime_agent_arn}")
print(f"エージェント ID: {runtime_agent_id}")
print(f"Runtime MCP URL: {agent_url}")

In [ ]:
# MCP サーバー認証用の OAuth2 認証情報プロバイダーを作成（Gateway → Runtime）

identity_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

cognito_provider = identity_client.create_oauth2_credential_provider(
    name=f"gateway-mcp-server-identity-{timestamp}",
    credentialProviderVendor="CustomOauth2",
    oauth2ProviderConfigInput={
        'customOauth2ProviderConfig': {
            'oauthDiscovery': {
                'discoveryUrl': runtime_cognito_discovery_url,
            },
            'clientId': runtime_client_id,
            'clientSecret': runtime_client_secret
        }
    }
)
cognito_provider_arn = cognito_provider['credentialProviderArn']
print("アウトバウンド OAuth2 認証情報プロバイダー ARN:", cognito_provider_arn)

## パート 7: Gateway ターゲットの作成と MCP サーバーの登録

AgentCore Gateway ターゲットを作成し、MCP サーバーを Gateway の背後にあるターゲットとして登録します。

In [ ]:
# MCP サーバーを指す Gateway ターゲットを作成

def create_gateway_target(gateway_id):
    """Gateway ターゲットを作成し、準備完了まで待機"""
    gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
    
    print("MCP ターゲットを作成中...")
    target_response = gateway_client.create_gateway_target(
        name=gateway_target_name,
        gatewayIdentifier=gateway_id,
        targetConfiguration={
            'mcp': {
                'mcpServer': {
                    'endpoint': agent_url
                }
            }
        },
        credentialProviderConfigurations=[
            {
                'credentialProviderType': 'OAUTH',
                'credentialProvider': {
                    'oauthCredentialProvider': {
                        'providerArn': cognito_provider_arn,
                        'scopes': [runtimeScopeString]
                    }
                }
            }
        ]
    )
    
    target_id = target_response['targetId']
    print(f"Gateway ターゲットを作成しました: {target_id}")
    
    # ターゲットが準備完了になるまで待機
    print("ターゲットが準備完了になるまで待機中...")
    while True:
        status_response = gateway_client.get_gateway_target(
            gatewayIdentifier=gateway_id,
            targetId=target_id
        )
        current_status = status_response.get('status', 'UNKNOWN')
        print(f"ターゲットステータス: {current_status}")
        if current_status == 'READY':
            print(f"ターゲットステータスレスポンス: {json.dumps(status_response, indent=2, default=str)}")
            break
        elif current_status == 'FAILED':
            print("ターゲットの作成に失敗しました！")
            print(f"失敗したターゲットの詳細: {json.dumps(status_response, indent=2, default=str)}")
            raise Exception("Target failed")
        time.sleep(10)
    
    print("ターゲットの準備が完了しました")
    return target_id

target_id = create_gateway_target(gateway_id)
print(f"\n✅ ターゲットの作成が完了しました: {target_id}")

## パート 8: FGAC のテスト

3 つのシナリオをテストします：

1. **FGAC によるツール呼び出し（リクエスト Gateway インターセプター）**
2. **FGAC によるセマンティック検索（レスポンス Gateway インターセプター）**
3. **FGAC によるツール一覧表示（レスポンス Gateway インターセプター）**

同じ Cognito ユーザープール + クライアントを使用しますが、異なるスコープでトークンをリクエストします：

- MCP ターゲット内のすべてのツールへのフルアクセス
- 単一のツール（例：`getOrder`）への限定アクセス

In [ ]:
def invoke_tool(gateway_url, access_token, tool_name, arguments=None):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
    # デフォルトの引数
    if arguments is None:
        arguments = {"orderId": 123} if tool_name != "getOrder" else {}
    
    payload = {
        "jsonrpc": "2.0",
        "id": "invoke-tool-request",
        "method": "tools/call",
        "params": {
            "name": f"{gateway_target_name}___{tool_name}",
            "arguments": arguments
        }
    }
    
    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json()

def get_access_token_for_scope(scope_label, scope):
    """
    指定されたスコープのアクセストークンを取得するヘルパー。
    Cognito がエラーを返した場合（access_token がない場合）、
    生のレスポンスを出力し、KeyError の代わりに明確な例外を発生させます。
    """
    print(f"\n[トークン] {scope_label} のトークンをリクエスト中")
    print(f"[トークン] 使用スコープ: {scope}")
    
    token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scope, REGION)
    print(f"[トークン] 生のトークンレスポンス:\n{json.dumps(token_response, indent=2)}")
    
    access_token = token_response.get("access_token")
    if not access_token:
        raise RuntimeError(
            f"'{scope_label}' のアクセストークンの取得に失敗しました。"
            f"レスポンスに 'access_token' が含まれていません。上記の [トークン] 出力を参照してください。"
        )
    return access_token

# 1) getOrder を getOrder スコープで → 許可
print("テスト 1: getOrder を getOrder スコープで - 許可されるべき")
scope = f"{RESOURCE_SERVER_ID}/{gateway_target_name}:getOrder"
token = get_access_token_for_scope("getOrder スコープ", scope)
result = invoke_tool(gateway_url, token, "getOrder")
print(json.dumps(result, indent=2))

# 2) updateOrder を getOrder スコープで → 拒否
print("\nテスト 2: updateOrder を getOrder スコープで - 拒否されるべき")
result = invoke_tool(gateway_url, token, "updateOrder")
print(json.dumps(result, indent=2))

# 3) deleteOrder を deleteOrder スコープで → 許可
print("\nテスト 3: deleteOrder を deleteOrder スコープで - 許可されるべき")
scope = f"{RESOURCE_SERVER_ID}/{gateway_target_name}:deleteOrder"
token = get_access_token_for_scope("deleteOrder スコープ", scope)
result = invoke_tool(gateway_url, token, "deleteOrder")
print(json.dumps(result, indent=2))

# 4) getOrder を deleteOrder スコープで → 拒否
print("\nテスト 4: getOrder を deleteOrder スコープで - 拒否されるべき")
result = invoke_tool(gateway_url, token, "getOrder")
print(json.dumps(result, indent=2))

# 5) フルアクセス（スコープ = MCP ターゲット）→ すべて許可
print("\nテスト 5: フルアクセススコープですべてのツール - すべて許可されるべき")
scope = f"{RESOURCE_SERVER_ID}/{gateway_target_name}"
token = get_access_token_for_scope("フルアクセススコープ", scope)

for tool in ["getOrder", "updateOrder", "cancelOrder", "deleteOrder"]:
    print(f"\n  {tool}:")
    result = invoke_tool(gateway_url, token, tool)
    print(json.dumps(result, indent=4))

In [ ]:
def semantic_search(gateway_url, access_token, query):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
    payload = {
        "jsonrpc": "2.0",
        "id": "semantic-search-request",
        "method": "tools/call",
        "params": {
            "name": "x_amz_bedrock_agentcore_search",
            "arguments": {
                "query": query
            }
        }
    }
    
    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json()

search_query = "注文をキャンセルおよび削除するためのツールを探してください"

# フルアクセススコープ
full_scope = f'{RESOURCE_SERVER_ID}/{gateway_target_name}'
full_token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, full_scope, REGION)
full_token = full_token_response["access_token"]

print("\n=== フルアクセススコープでのセマンティック検索 ===")
full_search_results = semantic_search(gateway_url, full_token, search_query)
print(json.dumps(full_search_results, indent=2))

# 限定スコープ（getOrder のみ）
limited_scope = f'{RESOURCE_SERVER_ID}/{gateway_target_name}:getOrder'
limited_token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, limited_scope, REGION)
limited_token = limited_token_response["access_token"]

print("\n=== 限定スコープ（getOrder のみ）でのセマンティック検索 ===")
limited_search_results = semantic_search(gateway_url, limited_token, search_query)
print(json.dumps(limited_search_results, indent=2))

# 結果の比較
print("\n=== セマンティック検索ツールの比較 ===")
full_tools = full_search_results.get('result', {}).get('structuredContent', {}).get('tools', [])
limited_tools = limited_search_results.get('result', {}).get('structuredContent', {}).get('tools', [])

print(f"フルアクセスツール数: {len(full_tools)}")
print(f"限定アクセスツール数: {len(limited_tools)}")
print(f"フルアクセスツール: {[tool.get('name') for tool in full_tools]}")
print(f"限定アクセスツール: {[tool.get('name') for tool in limited_tools]}")

In [ ]:
def list_tools(gateway_url, access_token):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
    payload = {
        "jsonrpc": "2.0",
        "id": str(uuid.uuid4()),
        "method": "tools/list",
        "params": {}  # MCP は params が存在することを期待（空でも可）
    }
    
    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json()

# フルアクセススコープ
full_scope = f'{RESOURCE_SERVER_ID}/{gateway_target_name}'
full_token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, full_scope, REGION)
full_token = full_token_response["access_token"]

print("\n=== フルアクセススコープでの MCP tools/list ===")
full_list_results = list_tools(gateway_url, full_token)
print(json.dumps(full_list_results, indent=2))

# 限定スコープ（getOrder のみ）
limited_scope = f'{RESOURCE_SERVER_ID}/{gateway_target_name}:getOrder'
limited_token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, limited_scope, REGION)
limited_token = limited_token_response["access_token"]

print("\n=== 限定スコープ（getOrder のみ）での MCP tools/list ===")
limited_list_results = list_tools(gateway_url, limited_token)
print(json.dumps(limited_list_results, indent=2))

# 結果の比較
print("\n=== TOOLS/LIST FGAC の比較 ===")
full_list_tools = full_list_results.get('result', {}).get('tools', [])
limited_list_tools = limited_list_results.get('result', {}).get('tools', [])

print(f"フルアクセスツール数: {len(full_list_tools)}")
print(f"限定アクセスツール数: {len(limited_list_tools)}")
print(f"フルアクセスツール: {[tool.get('name') for tool in full_list_tools]}")
print(f"限定アクセスツール: {[tool.get('name') for tool in limited_list_tools]}")

## パート 9: クリーンアップ – すべてのリソースを削除

> ⚠️ **警告:** このセクションは、このノートブックで作成されたリソースを**削除**します：
> - AgentCore Gateway と MCP ターゲット
> - Gateway インターセプター関数
> - Lambda と Gateway 用の IAM ロール
> - Runtime MCP サーバー
> - Cognito リソースは自動的に**削除されません**（手動クリーンアップを推奨）

In [ ]:
# Gateway ターゲットと Gateway を削除

print("\nGateway ターゲットと Gateway を削除中...")
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
utils.delete_gateway(gateway_client, gateway_id)

In [ ]:
# Lambda 関数を削除（リクエスト + レスポンスインターセプター）

print("\nLambda 関数を削除中...")

lambda_client = boto3.client('lambda', region_name=REGION)

# リクエストインターセプター
try:
    request_lambda_name = request_lambda_arn.split(":function:")[-1]
    lambda_client.delete_function(FunctionName=request_lambda_name)
    print(f"  ✓ リクエストインターセプターを削除しました: {request_lambda_name}")
except Exception as e:
    print(f"  ⚠ リクエストインターセプターの削除中にエラー: {e}")

# レスポンスインターセプター
try:
    response_lambda_name = response_lambda_arn.split(":function:")[-1]
    lambda_client.delete_function(FunctionName=response_lambda_name)
    print(f"  ✓ レスポンスインターセプターを削除しました: {response_lambda_name}")
except Exception as e:
    print(f"  ⚠ レスポンス Lambda の削除中にエラー: {e}")

In [ ]:
# IAM ロールを削除（Lambda + Gateway）

print("\nIAM ロールを削除中...")

# Gateway インターセプターロールをデタッチして削除
try:
    print("  Gateway インターセプターロールを削除中...")
    
    iam_client.detach_role_policy(
        RoleName=lambda_role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam_client.delete_role(RoleName=lambda_role_name)
    print(f"    ✓ Gateway インターセプターロールを削除しました: {lambda_role_name}")
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchEntity':
        print(f"    ⚠ ロールが見つかりません: {lambda_role_name}")
    else:
        print(f"    ⚠ Gateway インターセプターロールの削除中にエラー: {e}")

# Gateway ロールをデタッチして削除
try:
    print("  Gateway ロールを削除中...")
    
    iam_client.detach_role_policy(
        RoleName=gateway_role_name,
        PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
    )
    iam_client.delete_role(RoleName=gateway_role_name)
    print(f"    ✓ Gateway ロールを削除しました: {gateway_role_name}")
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchEntity':
        print(f"    ⚠ ロールが見つかりません: {gateway_role_name}")
    else:
        print(f"    ⚠ Gateway ロールの削除中にエラー: {e}")

In [ ]:
# MCP サーバーを削除

print("\nMCP サーバーを削除中...")

runtime_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
runtime_client.delete_agent_runtime(agentRuntimeId=runtime_agent_id)

In [ ]:
# アイデンティティプロバイダーを削除

print("\nアイデンティティプロバイダーを削除中...")

identity_client.delete_oauth2_credential_provider(name=f"gateway-mcp-server-identity-{timestamp}")

In [ ]:
print("\n" + "="*80)
print("  クリーンアップ完了")
print("="*80)

print("\n✓ 削除されたリソース:")
print(f"  • Gateway: {gateway_id}")
print(f"  • Gateway ターゲット: {target_id}")
print(f"  • リクエストインターセプター: {request_lambda_arn}")
print(f"  • レスポンスインターセプター: {response_lambda_arn}")
print(f"  • Lambda IAM ロール: {lambda_role_name}")
print(f"  • Gateway IAM ロール: {gateway_role_name}")
print(f"  • MCP サーバー: {runtime_agent_id}")

print("\n⚠️ Cognito ユーザープールはこのスクリプトでは削除されません。")
print("   必要に応じて AWS コンソールから手動で削除できます。")

print(f"\n📝 デプロイメントタイムスタンプ: {timestamp}")